In [1]:
import math
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random
import statistics

from SALib.sample import saltelli
from SALib.analyze import sobol

# Fix seed
np.random.seed(1234)


# Aquifer thickness b [m]
b = 10

# Hydraulic Conductivity from Cobuto, 1999 (adapted): 10^-4 - 10^-2 [m/s]
Kmin = 1e-04 * 86400
Kmax = 1e-02 * 86400

# Specific yield Sy from Johnson, 1967: 0.21 - 0.35 [-]
Sy_min = 0.21
Sy_max = 0.35

# Calculating D using eq. (2) from Sawyer et al., 2014
Dmin_temp = (Kmin*b)/Sy_max
Dmax_temp = (Kmax*b)/Sy_min

Dmin = math.log(Dmin_temp, 10) # lower bound for diffusivity
Dmax = math.log(Dmax_temp, 10) # upper bound for diffusivity
phi = 0 # phase

def solution(x, t, A, omega, D, phi):
    """This is the function which gives back the hydraulic head
    omega: frequency
    A: amplitude
    x: distance
    t: time"""
    sinus = math.sin(-x*math.sqrt(omega/(2*D))+ omega*t + phi)
    ret = A*math.exp(-x*math.sqrt(omega/(2*D)))*sinus
    return ret 


In [2]:
# increase
num_of_simulations = 200000

# vectors with 10000 samples of the parameters in matrix A & B
D_vector_A = [10**((Dmax - Dmin)*random.random() + Dmin) for _ in range(num_of_simulations)]
A_vector_A = [10**(random.random() - 1) for _ in range(num_of_simulations)]
omega_vector_A = [ (math.pi - 2*math.pi/7)*random.random() + 2*math.pi/7 for _ in range(num_of_simulations)]


D_vector_B = [10**((Dmax - Dmin)*random.random() + Dmin) for _ in range(num_of_simulations)]
A_vector_B = [10**(random.random() - 1) for _ in range(num_of_simulations)]
omega_vector_B = [ (math.pi - 2*math.pi/7)*random.random() + 2*math.pi/7 for _ in range(num_of_simulations)]

In [3]:
def calculate_solution(x, t, A_vector, omega_vector, D_vector):
    # return 10000 simulations of the hydraulic head
    y = []

    for idx in range(num_of_simulations):
        hydraulic_head = solution(x, t, A_vector[idx], omega_vector[idx], D_vector[idx], phi)
        y.append(hydraulic_head)
  
    return y

In [4]:
distances = [1, 10, 100] # given in meters
timesteps = [7, 30, 180] # given in days


for x in distances:
    for t in timesteps:

        y_A = calculate_solution(x, t, A_vector_A, omega_vector_A, D_vector_A)
        y_B = calculate_solution(x, t, A_vector_B, omega_vector_B, D_vector_B)
        y_C_1 = calculate_solution(x, t, A_vector_A, omega_vector_B, D_vector_B)
        y_C_2 = calculate_solution(x, t, A_vector_B, omega_vector_A, D_vector_B)
        y_C_3 = calculate_solution(x, t, A_vector_B, omega_vector_B, D_vector_A)


        avg = (sum(y_A)/num_of_simulations) * (sum(y_B)/num_of_simulations)
        
        # First order sensitivity indices
        
        S_1 = ((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_A, y_C_1)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg)
        S_2 = ((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_A, y_C_2)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg)
        S_3 = ((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_A, y_C_3)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg)

        print(x)
        print(t)

        print('S_A='+str(round(S_1,4)))
        print('S_omega='+str(round(S_2,4)))
        print('S_D='+str(round(S_3, 4)))
        print(" ")
        
        # Total order sensitivity indices
        
        ST_1 = 1 - (((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_B, y_C_1)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg))
        ST_2 = 1 - (((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_B, y_C_2)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg))
        ST_3 = 1 - (((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_B, y_C_3)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg))

        print('ST_A='+str(ST_1))
        print('ST_omega='+str(ST_2))
        print('ST_D='+str(ST_3))
        print(" ")

1
7
S_A=0.0097
S_omega=0.7031
S_D=0.0011
 
ST_A=0.3037700715162195
ST_omega=0.9897723100845142
ST_D=0.015061571071805058
 
1
30
S_A=-0.0032
S_omega=0.71
S_D=-0.0007
 
ST_A=0.2839307521133342
ST_omega=0.9998459672262898
ST_D=-0.0014034225913903686
 
1
180
S_A=0.0033
S_omega=0.7098
S_D=0.0019
 
ST_A=0.29183481144277545
ST_omega=0.9972497693088057
ST_D=0.008229171760042453
 
10
7
S_A=0.0083
S_omega=0.6733
S_D=0.001
 
ST_A=0.30264189599085156
ST_omega=0.9899263450537381
ST_D=0.05704133793640287
 
10
30
S_A=-0.0029
S_omega=0.6803
S_D=-0.0006
 
ST_A=0.28508005606247155
ST_omega=0.9984273146726971
ST_D=0.04185689576753682
 
10
180
S_A=0.0038
S_omega=0.68
S_D=0.0029
 
ST_A=0.2917406416142404
ST_omega=0.9967436423141584
ST_D=0.05034876333207439
 
100
7
S_A=0.0053
S_omega=0.2898
S_D=0.0119
 
ST_A=0.2917275108394465
ST_omega=0.9909811764755029
ST_D=0.5927595938439271
 
100
30
S_A=-0.0033
S_omega=0.3031
S_D=-0.0068
 
ST_A=0.28931374299556467
ST_omega=0.9902123481856453
ST_D=0.5757477884016389
 
10

In [17]:
import plotly.graph_objects as go

result = [[ None for _ in range(3)] for _ in range(3)]

result2 = [[ None for _ in range(3)] for _ in range(3)]

distances = [1, 10, 100] # given in meters
timesteps = [7, 30, 180] # given in days


for i, t in enumerate(timesteps): 
    for j, x in enumerate(distances):

        y_A = calculate_solution(x, t, A_vector_A, omega_vector_A, D_vector_A)
        y_B = calculate_solution(x, t, A_vector_B, omega_vector_B, D_vector_B)
        y_C_1 = calculate_solution(x, t, A_vector_A, omega_vector_B, D_vector_B)
        y_C_2 = calculate_solution(x, t, A_vector_B, omega_vector_A, D_vector_B)
        y_C_3 = calculate_solution(x, t, A_vector_B, omega_vector_B, D_vector_A)


        avg = (sum(y_A)/num_of_simulations) * (sum(y_B)/num_of_simulations)
        
        # First order sensitivity indices
        
        S_1 = ((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_A, y_C_1)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg)
        S_2 = ((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_A, y_C_2)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg)
        S_3 = ((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_A, y_C_3)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg)

        print(x)
        print(t)
        
        result[i][j] = [S_1, S_2, S_3]


        print('S_A='+str(S_1))
        print('S_omega='+str(S_2))
        print('S_D='+str(S_3))
        print(" ")
        
        # Total order sensitivity indices
        


        
        ST_1 = 1 - (((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_B, y_C_1)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg))
        ST_2 = 1 - (((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_B, y_C_2)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg))
        ST_3 = 1 - (((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_B, y_C_3)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg))

        
        result2[i][j] = [ST_1, ST_2, ST_3]

            
        print('ST_A='+str(ST_1))
        print('ST_omega='+str(ST_2))
        print('ST_D='+str(ST_3))
        print(" ")
    
x

1
7
S_A=0.00972268660906131
S_omega=0.7031051520861753
S_D=0.0011141640742122598
 
ST_A=0.3037700715162195
ST_omega=0.9897723100845142
ST_D=0.015061571071805058
 
10
7
S_A=0.0082904803508612
S_omega=0.6732542169978157
S_D=0.0009700062072582816
 
ST_A=0.30264189599085156
ST_omega=0.9899263450537381
ST_D=0.05704133793640287
 
100
7
S_A=0.0052948565776137395
S_omega=0.28980845756281404
S_D=0.011852547669913492
 
ST_A=0.2917275108394465
ST_omega=0.9909811764755029
ST_D=0.5927595938439271
 
1
30
S_A=-0.003195423261463409
S_omega=0.7100178004773283
S_D=-0.0007063022142699251
 
ST_A=0.2839307521133342
ST_omega=0.9998459672262898
ST_D=-0.0014034225913903686
 
10
30
S_A=-0.002881341016678868
S_omega=0.6803467381632774
S_D=-0.0005750443076031136
 
ST_A=0.28508005606247155
ST_omega=0.9984273146726971
ST_D=0.04185689576753682
 
100
30
S_A=-0.003303427780687081
S_omega=0.3031477754551598
S_D=-0.006774236675159831
 
ST_A=0.28931374299556467
ST_omega=0.9902123481856453
ST_D=0.5757477884016389
 
1
180

100

In [18]:
fig = make_subplots(rows=3, cols=3, horizontal_spacing = 0.12,vertical_spacing = 0.2, column_titles = ('x = 1', 'x = 10', 'x = 100'), row_titles=('t = 7', 't = 30', 't = 180')) # Plot histograms

fig.add_trace(go.Bar(x=["A", "Omega", "D"], y=result[0][0]), row=1, col=1)
fig.add_trace(go.Bar(x=["A", "Omega", "D"], y=result[0][1]),row=1, col=2)
fig.add_trace(go.Bar(x=["A", "Omega", "D"], y=result[0][2]), row=1, col=3)

fig.add_trace(go.Bar(x=["A", "Omega", "D"], y=result[1][0]), row=2, col=1)
fig.add_trace(go.Bar(x=["A", "Omega", "D"], y=result[1][1]), row=2, col=2)
fig.add_trace(go.Bar(x=["A", "Omega", "D"], y=result[1][2]), row=2, col=3)

fig.add_trace(go.Bar(x=["A", "Omega", "D"], y=result[2][0]), row=3, col=1)
fig.add_trace(go.Bar(x=["A", "Omega", "D"], y=result[2][1]), row=3, col=2)
fig.add_trace(go.Bar(x=["A", "Omega", "D"], y=result[2][2]), row=3, col=3)

fig.update_layout(title_text='First Order Sobol Indices', showlegend=False)




In [19]:
fig = make_subplots(rows=3, cols=3, horizontal_spacing = 0.12,vertical_spacing = 0.2, column_titles = ('x = 1', 'x = 10', 'x = 100'), row_titles=('t = 7', 't = 30', 't = 180')) # Plot histograms

fig.add_trace(go.Bar(x=["A", "Omega", "D"], y=result2[0][0]), row=1, col=1)
fig.add_trace(go.Bar(x=["A", "Omega", "D"], y=result2[0][1]),row=1, col=2)
fig.add_trace(go.Bar(x=["A", "Omega", "D"], y=result2[0][2]), row=1, col=3)

fig.add_trace(go.Bar(x=["A", "Omega", "D"], y=result2[1][0]), row=2, col=1)
fig.add_trace(go.Bar(x=["A", "Omega", "D"], y=result2[1][1]), row=2, col=2)
fig.add_trace(go.Bar(x=["A", "Omega", "D"], y=result2[1][2]), row=2, col=3)

fig.add_trace(go.Bar(x=["A", "Omega", "D"], y=result2[2][0]), row=3, col=1)
fig.add_trace(go.Bar(x=["A", "Omega", "D"], y=result2[2][1]), row=3, col=2)
fig.add_trace(go.Bar(x=["A", "Omega", "D"], y=result2[2][2]), row=3, col=3)

fig.update_layout(title_text='Total Order Sobol Indices', showlegend=False)


In [6]:
inpt = {
    'num_vars': 3,
    'names': ['A', 'omega', 'D'],
    'bounds': [[-1 , 0], # change to log bounds
               [(2*math.pi)/7, (2*math.pi)/2],
               [Dmin, Dmax]],
    'dists': ['unif', 'unif', 'unif'] 
}

param_values = saltelli.sample(inpt, N = 2**18, calc_second_order=True)

C:\Users\z004n49n\AppData\Local\Temp\ipykernel_15232\1361913787.py:10: DeprecationWarning:

`salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`



In [20]:
def return_hydraulic_head(values, x_input, t_input):
    Y = np.zeros([values.shape[0]])
    for i,X in enumerate(values):
        # change to log bounds
        Y[i] = (10**X[0] * math.exp(-x_input * math.sqrt(X[1] / (2*10**X[2])))) * (math.sin(-x_input * math.sqrt(X[1] / (2*10**X[2])) + ((X[1] * t_input))))  
    return Y

In [36]:
# save realizations
result3 = [[ None for _ in range(3)] for _ in range(3)]


for i, t in enumerate(timesteps): 
    for j, x in enumerate(distances):
        
        Y = return_hydraulic_head(param_values, x_input = np.array([x]), t_input = np.array([t]))
        Sobol = sobol.analyze(inpt, Y, print_to_console=True)
        S2 = Sobol["S2"]
        S2 = [x for row in S2 for x in row if x == x] # remove NaNs
        result3[i][j] = S2

             ST   ST_conf
A      0.296169  0.001888
omega  0.990374  0.007873
D      0.000510  0.000006
             S1   S1_conf
A      0.009627  0.003604
omega  0.703472  0.004640
D      0.000013  0.000106
                  S2   S2_conf
(A, omega)  0.286397  0.005992
(A, D)     -0.000004  0.006489
(omega, D)  0.000350  0.004292
[[            nan  2.86397018e-01 -3.90895848e-06]
 [            nan             nan  3.50092925e-04]
 [            nan             nan             nan]]
[0.2863970181596416, -3.908958477020322e-06, 0.0003500929248931775]
             ST   ST_conf
A      0.295754  0.001917
omega  0.990014  0.007768
D      0.043111  0.000475
             S1   S1_conf
A      0.009081  0.003927
omega  0.673541  0.004737
D      0.000671  0.000943
                  S2   S2_conf
(A, omega)  0.274206  0.005936
(A, D)      0.000203  0.006741
(omega, D)  0.029970  0.004305
[[           nan 2.74205710e-01 2.03326398e-04]
 [           nan            nan 2.99703552e-02]
 [           nan  

In [37]:
fig = make_subplots(rows=3, cols=3, horizontal_spacing = 0.12,vertical_spacing = 0.2, column_titles = ('x = 1', 'x = 10', 'x = 100'), row_titles=('t = 7', 't = 30', 't = 180')) # Plot histograms

fig.add_trace(go.Bar(x=["A, omega", "A, D", "omega, D"], y=result3[0][0]), row=1, col=1)
fig.add_trace(go.Bar(x=["A, omega", "A, D", "omega, D"], y=result3[0][1]), row=1, col=2)
fig.add_trace(go.Bar(x=["A, omega", "A, D", "omega, D"], y=result3[0][2]), row=1, col=3)

fig.add_trace(go.Bar(x=["A, omega", "A, D", "omega, D"], y=result3[1][0]), row=2, col=1)
fig.add_trace(go.Bar(x=["A, omega", "A, D", "omega, D"], y=result3[1][1]), row=2, col=2)
fig.add_trace(go.Bar(x=["A, omega", "A, D", "omega, D"], y=result3[1][2]), row=2, col=3)

fig.add_trace(go.Bar(x=["A, omega", "A, D", "omega, D"], y=result3[2][0]), row=3, col=1)
fig.add_trace(go.Bar(x=["A, omega", "A, D", "omega, D"], y=result3[2][1]), row=3, col=2)
fig.add_trace(go.Bar(x=["A, omega", "A, D", "omega, D"], y=result3[2][2]), row=3, col=3)

fig.update_layout(title_text='Second Order Sobol Indices', showlegend=False)
